In [ ]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../"))

from scripts.load_db import get_engine, create_schema, load_data

In [8]:
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "newami"
DB_USER = "postgres"  
DB_PASS = "1234"   


SCHEMA_PATH = "../scripts/schema.sql"
CLEANED_DATA = "../data/cleaned/bank_reviews_cleaned.csv"
ANALYZED_DATA = "../data/cleaned/task2_reviews_analyzed_bert.csv"

In [ ]:
engine = get_engine(DB_USER, DB_PASS, DB_HOST, DB_PORT, DB_NAME)

if engine:
    create_schema(engine, SCHEMA_PATH)

Successfully connected to database: newami
Applying schema from ../scripts/schema.sql...
Schema created/reset successfully.


In [10]:
if engine:
    load_data(engine, CLEANED_DATA, ANALYZED_DATA)

Loading datasets...
Merged Data Shape: (1140, 9)
Inserting reviews into PostgreSQL...
Success! 1140 rows inserted.


In [ ]:
if engine:
    print("--- Verifying Data Integrity ---")
    
    with engine.connect() as conn:
        count = pd.read_sql("SELECT COUNT(*) FROM reviews", conn)
        print(f"Total Reviews Stored: {count.iloc[0,0]}")
        
    query = """
    SELECT b.bank_name, COUNT(r.review_id) as count, AVG(r.sentiment_score) as avg_sentiment
    FROM reviews r
    JOIN banks b ON r.bank_id = b.bank_id
    GROUP BY b.bank_name
    """
    df_summary = pd.read_sql(query, engine)
    display(df_summary)

--- Verifying Data Integrity ---
Total Reviews Stored: 1140


,bank_name,count,avg_sentiment
0,BoA Mobile,391,0.956827
1,Dashen Bank,382,0.973861
2,Commercial Bank of Ethiopia,367,0.963389
